# Evaluation Pipeline

Here, I am implementing the metrices used for evaluating the evaluation quality

In [3]:
#imports 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
from sklearn.metrics.pairwise import cosine_similarity
import keras
import pickle
from PIL import Image
import lime
import torch
import io
import cv2
from scipy import ndimage

from lime import lime_image
from skimage.segmentation import mark_boundaries
import tensorflow as tf
from efficientnet.tfkeras import EfficientNetB4, EfficientNetB3
from tensorflow.keras.utils import load_img, img_to_array
from tensorflow.keras.applications.efficientnet import decode_predictions
from skimage.transform import resize  # Importing the resize function from scikit-image
from scipy.stats import mode

import keras
from transformers import DetrFeatureExtractor, DetrForSegmentation


from Utilities.utilities import *
from Utilities.lime_segmentation import *
from Utilities.xai_evaluation import *


In [4]:
date_path = "SAM_RES"

dataset_path = "./Dataset/Results/"+date_path+"/eval_results.pkl"
    
dataset = pickle.load(open(dataset_path, "rb"))

small_dataset= get_small_eval_dictionary(dataset)

In [5]:
lime_names = ["DSEG", "BayesLime", "SLIME", "GLIME", "LIME"]
images = np.unique(list((small_dataset.keys())))
for i in small_dataset.keys():
    domains = np.unique(list((small_dataset[i].keys())))
sorted_dictionary = {}

ground_truths = []
groundtruth_test = []

for local_domain in domains:
    sorted_dictionary[local_domain] = {}
    for local_image in images:
        for in_keys in list(small_dataset[local_image]["groundtruth"].keys()):
            if "ground" in in_keys.lower():
                ground_truths.append(small_dataset[local_image]["groundtruth"][in_keys][1])
            break
        for in_keys in list(small_dataset[local_image]["contrastivity"].keys()):
            if "groundtruth_test" in in_keys.lower():
                groundtruth_test.append(small_dataset[local_image]["contrastivity"][in_keys][1])
            break
        for local_name in small_dataset[local_image][local_domain].keys():
            #print(small_dataset[local_image])
            if local_image not in sorted_dictionary[local_domain].keys():
                sorted_dictionary[local_domain][local_image] = {}
            for local_metric in lime_names:
                if local_metric.lower() not in sorted_dictionary[local_domain][local_image].keys():
                    sorted_dictionary[local_domain][local_image][local_metric.lower()] = {}
            for local_metric in lime_names:
                if local_metric.lower() in local_name.lower():
                    sorted_dictionary[local_domain][local_image][local_metric.lower()][local_name] = small_dataset[local_image][local_domain][local_name]
                    
                    break


final_dict = {}
order_images = []
for i in list(sorted_dictionary.keys()):
    for domain in list(sorted_dictionary[i].keys()):
        domain_dict = {}
        local_dictionary = sorted_dictionary[i]
        for j in local_dictionary.keys():
            local_dict = local_dictionary[j]
            for k in local_dict.keys():
                if k not in domain_dict.keys():
                    domain_dict[k] = {}
                local_dict_ = local_dict[k]
                for l in local_dict_.keys():
                    if l not in domain_dict[k].keys():
                        domain_dict[k][l] = []
                    domain_dict[k][l].append(local_dict_[l])
                    
        final_dict[i] = domain_dict
    
eval_dict = {}
for i in final_dict.keys():
    local_df = {}
    for j in final_dict[i].keys():
        for k in final_dict[i][j].keys():
            if len(np.array(final_dict[i][j][k]).shape) == 2:
                if "preservation_noise" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] == ground_truths[num]:
                            local_df[k] += 1
                elif "deletion_noise" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] != ground_truths[num]:
                            local_df[k] += 1
                
                elif "preservation_check_contrast" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] == groundtruth_test[num]:
                            local_df[k] += 1
                elif "deletion_check_contrast" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] != groundtruth_test[num]:
                            local_df[k] += 1
                
                elif "preservation" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] == ground_truths[num]:
                            local_df[k] += 1

                elif "single_deletion" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] == ground_truths[num]:
                            local_df[k] += 1
                elif "deletion" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] != ground_truths[num]:
                            local_df[k] += 1
                elif "model_random" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] != ground_truths[num]:
                            local_df[k] += 1
                elif "eval_random" in k:
                    local_df[k] = 0
                    for num, data in enumerate(final_dict[i][j][k]):
                        if data[1] != ground_truths[num]:
                            local_df[k] += 1
            else:
                try:
                    local_df[k] = np.mean(final_dict[i][j][k])
                except:
                    pass
                
    eval_dict[i] = local_df

def dataframe_results(data):
    df = pd.DataFrame(data.items(), columns=['Metric', 'Value'])

    # Splitting the 'Metric' column to extract additional categorical information
    df[['Method', 'Metric_Type']] = df['Metric'].str.split('_', n=1, expand=True)

    # Drop the original 'Metric' column as its information has been extracted
    df.drop('Metric', axis=1, inplace=True)

    # Reordering the columns for better readability
    df = df[['Method', 'Metric_Type', 'Value']]

    df_pivot = df.pivot(index='Metric_Type', columns='Method', values='Value')

    # Display the pivoted DataFrame
    return df_pivot
    
eval_dict.keys()

dict_keys(['consistency', 'contrastivity', 'correctness', 'groundtruth', 'output_completness'])

## Overview

In [6]:
dataframe_results(eval_dict["groundtruth"])

Method,BayesLIME,DSEG,GLIME,LIME,SLIME
Metric_Type,,,,,
segments,15.85000,15.850000,15.850000,16.70000,15.85000
time,52.25237,45.743605,48.349895,19.64428,49.44002


In [7]:
df_color = ColoredDF()
df_color.create_pandas_plot(small_dataset, to_save_path = './Dataset/Results/'+date_path+'/', id = 0)


,Groundtruth,DSEG_time,DSEG_segments,LIME_time,LIME_segments,BayesLIME_time,BayesLIME_segments,SLIME_time,SLIME_segments,GLIME_time,GLIME_segments
000000521141.jpg,['n06794110' 'street_sign' '0.14786814'],39.961400,18.000000,24.786300,17.000000,52.649400,18.000000,47.033100,18.000000,49.428200,18.000000
000000000785.jpg,['n04228054' 'ski' '0.9835006'],45.293300,5.000000,22.313700,7.000000,45.982000,5.000000,45.985500,5.000000,42.537100,5.000000
000000289059.jpg,['n04039381' 'racket' '0.9071478'],56.690400,7.000000,24.963500,8.000000,51.088500,7.000000,53.412100,7.000000,57.546800,7.000000
n02134084_ice_bear.JPEG,['n02134084' 'ice_bear' '0.52830356'],41.295900,11.000000,25.299100,10.000000,41.379600,11.000000,41.669200,11.000000,45.441100,11.000000
n03891251_park_bench.JPEG,['n03891251' 'park_bench' '0.59677887'],47.844100,14.000000,14.402800,16.000000,44.935300,14.000000,39.793800,14.000000,41.602600,14.000000
n02480855_gorilla.JPEG,['n02480855' 'gorilla' '0.2135582'],57.643900,31.000000,19.054500,35.000000,92.002200,31.000000,86.959000,31.000000,91.447500,31.000000
n02988304_CD_player.JPEG,['n02988304' 'CD_player' '0.33121333'],39.711700,7.000000,29.118600,9.000000,51.045500,7.000000,48.777600,7.000000,46.079700,7.000000
000000290619.jpg,['n03133878' 'Crock_Pot' '0.104617536'],51.917800,5.000000,21.628000,8.000000,47.605600,5.000000,44.639100,5.000000,42.468700,5.000000
n04008634_projectile.JPEG,['n04008634' 'projectile' '0.5816481'],59.410300,36.000000,21.569000,36.000000,49.225700,36.000000,47.256500,36.000000,46.405200,36.000000
n02981792_catamaran.JPEG,['n02981792' 'catamaran' '0.64228153'],33.726800,10.000000,11.116400,14.000000,46.977000,10.000000,44.055400,10.000000,46.427600,10.000000


## Correctness

In [8]:
dataframe_results(eval_dict["correctness"])

Method,BayesLIME,DSEG,GLIME,LIME,SLIME
Metric_Type,,,,,
Compactness,0.178915,0.235865,0.213140,0.147295,0.204250
incremental_deletion,0.294040,0.261495,0.112805,0.293715,0.086825
prediction_eval_random,19.000000,18.000000,20.000000,18.000000,19.000000
prediction_model_random,16.000000,18.000000,17.000000,18.000000,18.000000
prediction_single_deletion,5.000000,4.000000,3.000000,5.000000,3.000000


In [9]:
df_color.create_pandas_plot(small_dataset, to_save_path = './Dataset/Results/'+date_path+'/', id = 1)

,DSEG_prediction_model_random,LIME_prediction_model_random,BayesLIME_prediction_model_random,SLIME_prediction_model_random,GLIME_prediction_model_random,DSEG_prediction_eval_random,LIME_prediction_eval_random,BayesLIME_prediction_eval_random,SLIME_prediction_eval_random,GLIME_prediction_eval_random,DSEG_incremental_deletion,LIME_incremental_deletion,BayesLIME_incremental_deletion,SLIME_incremental_deletion,GLIME_incremental_deletion,DSEG_prediction_single_deletion,DSEG_Compactness,LIME_prediction_single_deletion,LIME_Compactness,BayesLIME_prediction_single_deletion,BayesLIME_Compactness,SLIME_prediction_single_deletion,SLIME_Compactness,GLIME_prediction_single_deletion,GLIME_Compactness,Groundtruth,DSEG_time,DSEG_segments,LIME_time,LIME_segments,BayesLIME_time,BayesLIME_segments,SLIME_time,SLIME_segments,GLIME_time,GLIME_segments
000000521141.jpg,['n02777292' 'balance_beam' '0.0385846'],['n02974003' 'car_wheel' '0.07115947'],['n03532672' 'hook' '0.026524968'],['n03532672' 'hook' '0.026524968'],['n03498962' 'hatchet' '0.041639976'],['n04153751' 'screw' '0.02919803'],['n03467068' 'guillotine' '0.03927183'],['n03532672' 'hook' '0.022448368'],['n01930112' 'nematode' '0.030657671'],['n01930112' 'nematode' '0.030657671'],0.034400,0.000000,0.000000,0.000000,0.035500,['n09332890' 'lakeside' '0.14000916'],0.165900,['n09421951' 'sandbar' '0.09547448'],0.192600,['n09332890' 'lakeside' '0.14000916'],0.165900,['n03976657' 'pole' '0.12589882'],0.212800,['n09332890' 'lakeside' '0.14000916'],0.165900,['n06794110' 'street_sign' '0.14786814'],39.961400,18.000000,24.786300,17.000000,52.649400,18.000000,47.033100,18.000000,49.428200,18.000000
000000000785.jpg,['n04228054' 'ski' '0.079179265'],['n04228054' 'ski' '0.0461742'],['n04228054' 'ski' '0.87204546'],['n03804744' 'nail' '0.07187442'],['n03291819' 'envelope' '0.056568306'],['n04228054' 'ski' '0.9094546'],['n03291819' 'envelope' '0.41522413'],['n03804744' 'nail' '0.07187442'],['n04228054' 'ski' '0.87204546'],['n03291819' 'envelope' '0.58706206'],0.130200,0.750600,0.024900,0.000000,0.000000,['n04228054' 'ski' '0.49055734'],0.630300,['n02894605' 'breakwater' '0.10114555'],0.207100,['n04228054' 'ski' '0.49055734'],0.630300,['n04228054' 'ski' '0.49055734'],0.630300,['n04228054' 'ski' '0.49055734'],0.630300,['n04228054' 'ski' '0.9835006'],45.293300,5.000000,22.313700,7.000000,45.982000,5.000000,45.985500,5.000000,42.537100,5.000000
000000289059.jpg,['n03291819' 'envelope' '0.08861872'],['n04039381' 'racket' '0.03092892'],['n04039381' 'racket' '0.28504896'],['n04039381' 'racket' '0.28504896'],['n03041632' 'cleaver' '0.04533817'],['n02977058' 'cash_machine' '0.066882856'],['n03291819' 'envelope' '0.10280368'],['n04039381' 'racket' '0.051801126'],['n03594734' 'jean' '0.09358135'],['n03594734' 'jean' '0.09358135'],0.586400,0.122000,0.000000,0.000000,0.000000,['n03873416' 'paddle' '0.08267101'],0.293900,['n04039381' 'racket' '0.29108208'],0.160600,['n04039381' 'racket' '0.12964143'],0.057300,['n03873416' 'paddle' '0.08267101'],0.293900,['n04039381' 'racket' '0.12964143'],0.057300,['n04039381' 'racket' '0.9071478'],56.690400,7.000000,24.963500,8.000000,51.088500,7.000000,53.412100,7.000000,57.546800,7.000000
n02134084_ice_bear.JPEG,['n03146219' 'cuirass' '0.04960224'],['n04522168' 'vase' '0.026625821'],['n02951585' 'can_opener' '0.062126614'],['n03041632' 'cleaver' '0.031222275'],['n03146219' 'cuirass' '0.04960224'],['n03146219' 'cuirass' '0.02882185'],['n03291819' 'envelope' '0.05009985'],['n04208210' 'shovel' '0.053567838'],['n03958227' 'plastic_bag' '0.07008829'],['n03666591' 'lighter' '0.027904764'],0.103200,0.440500,0.866100,0.000000,0.479200,['n02134084' 'ice_bear' '0.050765876'],0.396100,['n02894605' 'breakwater' '0.25962472'],0.281300,['n02134084' 'ice_bear' '0.050765876'],0.396100,['n02134084' 'ice_bear' '0.21139362'],0.149000,['n02894605' 'breakwater' '0.060918905'],0.175200,['n02134084' 'ice_bear' '0.52830356'],41.295900,11.000000,25.299100,10.000000,41.379600,11.000000,41.669200,11.000000,45

## Contrastivity


In [10]:
dataframe_results(eval_dict["contrastivity"])

Method,BayesLIME,DSEG,GLIME,LIME,SLIME
Metric_Type,,,,,
deletion_check_contrast,10,6,7,5,10
deletion_noise_check,9,10,9,7,11
preservation_check_contrast,9,6,8,7,12
preservation_noise_check,8,6,8,9,10
single_deletion,7,7,7,5,7


In [11]:

df_color.create_pandas_plot(small_dataset, to_save_path = './Dataset/Results/'+date_path+'/', id = 2)

,Groundtruth_test,DSEG_preservation_check_contrast,DSEG_deletion_check_contrast,LIME_preservation_check_contrast,LIME_deletion_check_contrast,BayesLIME_preservation_check_contrast,BayesLIME_deletion_check_contrast,SLIME_preservation_check_contrast,SLIME_deletion_check_contrast,GLIME_preservation_check_contrast,GLIME_deletion_check_contrast,DSEG_single_deletion,LIME_single_deletion,BayesLIME_single_deletion,SLIME_single_deletion,GLIME_single_deletion,DSEG_preservation_noise_check,DSEG_deletion_noise_check,LIME_preservation_noise_check,LIME_deletion_noise_check,BayesLIME_preservation_noise_check,BayesLIME_deletion_noise_check,SLIME_preservation_noise_check,SLIME_deletion_noise_check,GLIME_preservation_noise_check,GLIME_deletion_noise_check,Groundtruth,DSEG_time,DSEG_segments,LIME_time,LIME_segments,BayesLIME_time,BayesLIME_segments,SLIME_time,SLIME_segments,GLIME_time,GLIME_segments
000000521141.jpg,['n06794110' 'street_sign' '0.1489823'],['n06874185' 'traffic_light' '0.33398026'],['n06794110' 'street_sign' '0.14456776'],['n04037443' 'racer' '0.1498511'],['n06794110' 'street_sign' '0.11443315'],['n06874185' 'traffic_light' '0.33398026'],['n06794110' 'street_sign' '0.14456776'],['n03976657' 'pole' '0.32328492'],['n03891332' 'parking_meter' '0.17093869'],['n06874185' 'traffic_light' '0.33398026'],['n06794110' 'street_sign' '0.14456776'],['n09332890' 'lakeside' '0.34543875'],['n09421951' 'sandbar' '0.64605445'],['n09332890' 'lakeside' '0.34543875'],['n03976657' 'pole' '0.4652459'],['n09332890' 'lakeside' '0.34543875'],['n02930766' 'cab' '0.29100904'],['n02930766' 'cab' '0.29337314'],['n03976657' 'pole' '0.18583259'],['n06874185' 'traffic_light' '0.13524109'],['n06874185' 'traffic_light' '0.24283795'],['n06794110' 'street_sign' '0.11432691'],['n06874185' 'traffic_light' '0.24283795'],['n06794110' 'street_sign' '0.11432691'],['n06874185' 'traffic_light' '0.24283795'],['n06794110' 'street_sign' '0.11432691'],['n06794110' 'street_sign' '0.14786814'],39.961400,18.000000,24.786300,17.000000,52.649400,18.000000,47.033100,18.000000,49.428200,18.000000
000000000785.jpg,['n04228054' 'ski' '0.8964271'],['n04228054' 'ski' '0.91820824'],['n04228054' 'ski' '0.79289573'],['n04228054' 'ski' '0.68936986'],['n04228054' 'ski' '0.90667474'],['n04228054' 'ski' '0.91820824'],['n04228054' 'ski' '0.79289573'],['n04228054' 'ski' '0.91820824'],['n04228054' 'ski' '0.79289573'],['n04228054' 'ski' '0.91820824'],['n04228054' 'ski' '0.79289573'],['n04228054' 'ski' '0.7284479'],['n04228054' 'ski' '0.14062943'],['n04228054' 'ski' '0.7284479'],['n04228054' 'ski' '0.7284479'],['n04228054' 'ski' '0.7284479'],['n04228054' 'ski' '0.9374531'],['n04228054' 'ski' '0.6128504'],['n04228054' 'ski' '0.4718929'],['n04228054' 'ski' '0.92839676'],['n04228054' 'ski' '0.93043053'],['n04228054' 'ski' '0.6538172'],['n04228054' 'ski' '0.93043053'],['n04228054' 'ski' '0.6538172'],['n04228054' 'ski' '0.93043053'],['n04228054' 'ski' '0.6538172'],['n04228054' 'ski' '0.9835006'],45.293300,5.000000,22.313700,7.000000,45.982000,5.000000,45.985500,5.000000,42.537100,5.000000
000000289059.jpg,['n04039381' 'racket' '0.8029209'],['n04039381' 'racket' '0.36401093'],['n04039381' 'racket' '0.84473366'],['n04039381' 'racket' '0.8433766'],['n04039381' 'racket' '0.81327224'],['n04039381' 'racket' '0.8551795'],['n04039381' 'racket' '0.58829564'],['n04039381' 'racket' '0.36401093'],['n04039381' 'racket' '0.84473366'],['n04039381' 'racket' '0.8551795'],['n04039381' 'racket' '0.58829564'],['n02641379' 'gar' '0.10539715'],['n04039381' 'racket' '0.70222837'],['n04039381' 'racket' '0.47391814'],['n02641379' 'gar' '0.10539715'],['n04039381' 'racket' '0.47391814'],['n04039381' 'racket' '0.72867966'],['n04039381' 'racket' '0.7569705'],['n04039381' 'racket' '0.8268946'],['n04039381' 'racket' '0.93042606'],['n04039381' 'racket' '0.49117103'],['n04039381' 'racket' '0.88417673'],['n04039381' 'racket' '0.824431'],['n04039381' 'racket' '0.59229475'],['n04039381' 'racket' '0.824431'],['n04039381' 'racket' '0.59229475

## Output Completenes


In [12]:
dataframe_results(eval_dict["output_completness"])

Method,BayesLIME,DSEG,GLIME,LIME,SLIME
Metric_Type,,,,,
deletion_check,13,10,9,10,12
preservation_check,7,7,6,7,9


In [13]:
df_color.create_pandas_plot(small_dataset, to_save_path = './Dataset/Results/'+date_path+'/', id = 3)

,DSEG_preservation_check,DSEG_deletion_check,LIME_preservation_check,LIME_deletion_check,BayesLIME_preservation_check,BayesLIME_deletion_check,SLIME_preservation_check,SLIME_deletion_check,GLIME_preservation_check,GLIME_deletion_check,Groundtruth,DSEG_time,DSEG_segments,LIME_time,LIME_segments,BayesLIME_time,BayesLIME_segments,SLIME_time,SLIME_segments,GLIME_time,GLIME_segments
000000521141.jpg,['n03126707' 'crane' '0.047239758'],['n06794110' 'street_sign' '0.09605626'],['n02974003' 'car_wheel' '0.07115947'],['n06794110' 'street_sign' '0.089145295'],['n03126707' 'crane' '0.047239758'],['n06794110' 'street_sign' '0.09605626'],['n03498962' 'hatchet' '0.041639976'],['n03837869' 'obelisk' '0.1022912'],['n03126707' 'crane' '0.047239758'],['n06794110' 'street_sign' '0.09605626'],['n06794110' 'street_sign' '0.14786814'],39.961400,18.000000,24.786300,17.000000,52.649400,18.000000,47.033100,18.000000,49.428200,18.000000
000000000785.jpg,['n04228054' 'ski' '0.87204546'],['n04228054' 'ski' '0.5836076'],['n04228054' 'ski' '0.0461742'],['n04228054' 'ski' '0.96101725'],['n04228054' 'ski' '0.87204546'],['n04228054' 'ski' '0.5836076'],['n04228054' 'ski' '0.87204546'],['n04228054' 'ski' '0.5836076'],['n04228054' 'ski' '0.87204546'],['n04228054' 'ski' '0.5836076'],['n04228054' 'ski' '0.9835006'],45.293300,5.000000,22.313700,7.000000,45.982000,5.000000,45.985500,5.000000,42.537100,5.000000
000000289059.jpg,['n04039381' 'racket' '0.051801126'],['n04039381' 'racket' '0.7984358'],['n04039381' 'racket' '0.722895'],['n04039381' 'racket' '0.4810981'],['n04039381' 'racket' '0.28504896'],['n04039381' 'racket' '0.65856916'],['n04039381' 'racket' '0.051801126'],['n04039381' 'racket' '0.7984358'],['n04039381' 'racket' '0.28504896'],['n04039381' 'racket' '0.65856916'],['n04039381' 'racket' '0.9071478'],56.690400,7.000000,24.963500,8.000000,51.088500,7.000000,53.412100,7.000000,57.546800,7.000000
n02134084_ice_bear.JPEG,['n03467068' 'guillotine' '0.027893737'],['n02134084' 'ice_bear' '0.13813636'],['n02231487' 'walking_stick' '0.07650201'],['n02134084' 'ice_bear' '0.68051416'],['n03146219' 'cuirass' '0.04960224'],['n02120079' 'Arctic_fox' '0.15261398'],['n02134084' 'ice_bear' '0.32857126'],['n01695060' 'Komodo_dragon' '0.030753268'],['n03773504' 'missile' '0.050103594'],['n02134084' 'ice_bear' '0.6676824'],['n02134084' 'ice_bear' '0.52830356'],41.295900,11.000000,25.299100,10.000000,41.379600,11.000000,41.669200,11.000000,45.441100,11.000000
n03891251_park_bench.JPEG,['n03743016' 'megalith' '0.240824'],['n04355338' 'sundial' '0.064607404'],['n03794056' 'mousetrap' '0.053915996'],['n09332890' 'lakeside' '0.15059836'],['n04355338' 'sundial' '0.30354115'],['n09332890' 'lakeside' '0.23971914'],['n04355338' 'sundial' '0.30354115'],['n09332890' 'lakeside' '0.23971914'],['n03743016' 'megalith' '0.05633293'],['n03891251' 'park_bench' '0.094857566'],['n03891251' 'park_bench' '0.59677887'],47.844100,14.000000,14.402800,16.000000,44.935300,14.000000,39.793800,14.000000,41.602600,14.000000
n02480855_gorilla.JPEG,['n02667093' 'abaya' '0.025448188'],['n02487347' 'macaque' '0.13328512'],['n02480855' 'gorilla' '0.19154902'],['n01704323' 'triceratops' '0.046408672'],['n01608432' 'kite' '0.06648098'],['n01616318' 'vulture' '0.04566874'],['n02879718' 'bow' '0.045010168'],['n02480855' 'gorilla' '0.16766116'],['n02667093' 'abaya' '0.035279524'],['n02487347' 'macaque' '0.1203639'],['n02480855' 'gorilla' '0.2135582'],57.643900,31.000000,19.054500,35.000000,92.002200,31.000000,86.959000,31.000000,91.447500,31.000000
n02988304_CD_player.JPEG,['n03623198' 'knee_pad' '0.1735888'],['n02988304' 'CD_player' '0.22544907'],['n04127249' 'safety_pin' '0.07940575'],['n02988304' 'CD_player' '0.25285202'],['n02988304' 'CD_player' '0.15785496'],['n02910353' 'buckle' '0.10194838'],['n02988304' 'CD_player' '0.15785496'],['n02910353' 'buckle' '0.10194838'],['n02988304' 'CD_player' '0.15785496'],['n02910353' 'buckle' '0.10194838'],['n02988304' 'CD_player' '0.33121333'],39.711700,7.000000,29.118600,9

## Consistency

In [14]:
dataframe_results(eval_dict["consistency"])

Method,BayesLIME,DSEG,GLIME,LIME,SLIME
Metric_Type,,,,,
deletion_noise_check,9.00000,12.000000,9.0000,6.000000,12.00000
preservation_noise_check,6.00000,8.000000,6.0000,6.000000,10.00000
stability,0.02244,0.023545,0.0196,0.021315,0.02263


In [15]:
df_color.create_pandas_plot(small_dataset, to_save_path = './Dataset/Results/'+date_path+'/', id = 4)

,DSEG_stability,LIME_stability,BayesLIME_stability,SLIME_stability,GLIME_stability,DSEG_preservation_noise_check,DSEG_deletion_noise_check,LIME_preservation_noise_check,LIME_deletion_noise_check,BayesLIME_preservation_noise_check,BayesLIME_deletion_noise_check,SLIME_preservation_noise_check,SLIME_deletion_noise_check,GLIME_preservation_noise_check,GLIME_deletion_noise_check,Groundtruth,DSEG_time,DSEG_segments,LIME_time,LIME_segments,BayesLIME_time,BayesLIME_segments,SLIME_time,SLIME_segments,GLIME_time,GLIME_segments
000000521141.jpg,0.008300,0.015000,0.014200,0.008700,0.007200,['n02825657' 'bell_cote' '0.05756829'],['n02930766' 'cab' '0.053916644'],['n03291819' 'envelope' '0.047911674'],['n03837869' 'obelisk' '0.10873566'],['n03126707' 'crane' '0.054534417'],['n06794110' 'street_sign' '0.07245696'],['n03126707' 'crane' '0.054534417'],['n06794110' 'street_sign' '0.07245696'],['n03126707' 'crane' '0.054534417'],['n06794110' 'street_sign' '0.07245696'],['n06794110' 'street_sign' '0.14786814'],39.961400,18.000000,24.786300,17.000000,52.649400,18.000000,47.033100,18.000000,49.428200,18.000000
000000000785.jpg,0.022200,0.024400,0.023900,0.015800,0.017500,['n04228054' 'ski' '0.9356242'],['n04228054' 'ski' '0.32868195'],['n04228054' 'ski' '0.07051429'],['n04228054' 'ski' '0.93029207'],['n04228054' 'ski' '0.8925803'],['n04228054' 'ski' '0.57093245'],['n04228054' 'ski' '0.8925803'],['n04228054' 'ski' '0.57093245'],['n04228054' 'ski' '0.8925803'],['n04228054' 'ski' '0.57093245'],['n04228054' 'ski' '0.9835006'],45.293300,5.000000,22.313700,7.000000,45.982000,5.000000,45.985500,5.000000,42.537100,5.000000
000000289059.jpg,0.038700,0.034600,0.041400,0.032200,0.030100,['n04039381' 'racket' '0.22215234'],['n04039381' 'racket' '0.5306591'],['n04039381' 'racket' '0.20364122'],['n04039381' 'racket' '0.7639315'],['n04409515' 'tennis_ball' '0.04553536'],['n04039381' 'racket' '0.7190372'],['n04409515' 'tennis_ball' '0.2599623'],['n04039381' 'racket' '0.6554663'],['n04409515' 'tennis_ball' '0.2599623'],['n04039381' 'racket' '0.6554663'],['n04039381' 'racket' '0.9071478'],56.690400,7.000000,24.963500,8.000000,51.088500,7.000000,53.412100,7.000000,57.546800,7.000000
n02134084_ice_bear.JPEG,0.023400,0.020700,0.027300,0.030700,0.026300,['n02134084' 'ice_bear' '0.9506539'],['n02231487' 'walking_stick' '0.07253891'],['n02134084' 'ice_bear' '0.22482087'],['n02134084' 'ice_bear' '0.33534867'],['n03146219' 'cuirass' '0.036007416'],['n02134084' 'ice_bear' '0.31500247'],['n02134084' 'ice_bear' '0.5347101'],['n01695060' 'Komodo_dragon' '0.03829281'],['n02134084' 'ice_bear' '0.5347101'],['n01695060' 'Komodo_dragon' '0.03829281'],['n02134084' 'ice_bear' '0.52830356'],41.295900,11.000000,25.299100,10.000000,41.379600,11.000000,41.669200,11.000000,45.441100,11.000000
n03891251_park_bench.JPEG,0.022700,0.022800,0.024100,0.024400,0.020100,['n03891251' 'park_bench' '0.061373897'],['n09332890' 'lakeside' '0.12688455'],['n09332890' 'lakeside' '0.13690421'],['n03891251' 'park_bench' '0.48251855'],['n03891251' 'park_bench' '0.061373897'],['n09332890' 'lakeside' '0.12688455'],['n03891251' 'park_bench' '0.061373897'],['n09332890' 'lakeside' '0.12688455'],['n03743016' 'megalith' '0.0666676'],['n03891251' 'park_bench' '0.08968832'],['n03891251' 'park_bench' '0.59677887'],47.844100,14.000000,14.402800,16.000000,44.935300,14.000000,39.793800,14.000000,41.602600,14.000000
n02480855_gorilla.JPEG,0.015200,0.012100,0.014500,0.016500,0.013800,['n03804744' 'nail' '0.03160894'],['n02488291' 'langur' '0.1243151'],['n02480855' 'gorilla' '0.0934084'],['n01843383' 'toucan' '0.04746608'],['n02879718' 'bow' '0.08749975'],['n02480855' 'gorilla' '0.21628855'],['n02441942' 'weasel' '0.04239898'],['n03000684' 'chain_saw' '0.058157343'],['n03532672' 'hook' '0.047052853'],['n02480855' 'gorilla' '0.13180207'],['n02480855' 'gorilla' '0.2135582'],57.643900,31.000000,19.054500,35.000000,92.002200,31.000000,86.959000,31.000000,91.447500,31.000000
n02988304_CD_player.JPEG,0.026800,0.030100,0.032800,0.